In [1]:
import pandas as pd 

In [2]:
corpus_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTest.csv")
#corpus_test

In [3]:
corpus_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTrain.csv")
#corpus_train

In [16]:
corpus_test['Content'] = corpus_test['Content'].str.strip()
corpus_test['Content'] = corpus_test['Content'].str.lower()

corpus_train['Content'] = corpus_train['Content'].str.strip()
corpus_train['Content'] = corpus_train['Content'].str.lower()

In [17]:
spec_chars = ["!", '"', "#", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<",
              "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~", "–"]

for char in spec_chars:
    corpus_test['Content'] = corpus_test['Content'].str.replace(char, '')

for char in spec_chars:
    corpus_train['Content'] = corpus_train['Content'].str.replace(char, '')

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 1000)
vectorizer.fit(corpus_test['Content'])
vectorized_test_content = vectorizer.transform(corpus_test['Content'])
print(vectorized_test_content.shape)

(5374, 1000)


In [6]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 1000)
vectorizer.fit(corpus_train['Content'])
vectorized_train_content = vectorizer.transform(corpus_train['Content'])
print(vectorized_train_content.shape)

(531990, 1000)


In [7]:
corpus_compined = pd.DataFrame(corpus_test['Content'].append(corpus_train['Content'], ignore_index = True)) 
corpus_compined['Content']

0                     How do I get good marks in college?\n
1         Can an android app use SMS only to communicate...
2         What small detail from an Indian movie do you ...
3         Why can not Hindu women be the soldier of Hind...
4         How would you write out twelve lakh twelve tho...
                                ...                        
537359                              Why is SEO important?\n
537360    Who is the best employer for Robotic Process a...
537361    Is it possible to cure the holes caused by pim...
537362              How many employees does Infosys have?\n
537363    What is the maximum size for an attachment in ...
Name: Content, Length: 537364, dtype: object

In [ ]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 1000)
vectorizer.fit(corpus_compined['Content'])
vectorized_combined_content = vectorizer.transform(corpus_compined['Content'])
print(vectorized_combined_content.shape)
print(vectorized_combined_content.toarray()[0])

Exact Cosine Similarity

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
#need to be applied for every train row, index >= 5374
c = cosine_similarity(vectorized_combined_content[0], vectorized_combined_content[5374])
print(c)

[[0.]]


Exact Jaccard Similarity

In [11]:
import numpy as np
from sklearn.metrics import jaccard_score

In [20]:
temp_arr = vectorized_combined_content.toarray()
c = jaccard_score(temp_arr[5374], temp_arr[0])
print(c)

0.0


MinHash LSH

In [ ]:
!pip install --upgrade datasketch

In [15]:
from datasketch import MinHash, MinHashLSH

In [ ]:
#minhash before vectorization 
'''
m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in set1:
    m1.update(d.encode('utf8'))
for d in set2:
    m2.update(d.encode('utf8'))
for d in set3:
    m3.update(d.encode('utf8'))

# Create LSH index
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)
print("Approximate neighbours with Jaccard similarity > 0.5", result)
'''
